In [0]:
%sql
USE CATALOG workspace;

In [0]:
%sql
DROP DATABASE IF EXISTS bricks_hub_olist CASCADE;

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS bricks_hub_olist;

In [0]:
%sql
USE DATABASE bricks_hub_olist;

In [0]:
%sql
-- BRONZE: Orders 
DROP TABLE IF EXISTS workspace.bricks_hub_olist.bronze_orders;

In [0]:
%fs ls /Volumes/course_ws_practise/default/olist

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.bricks_hub_olist.bronze_orders AS
SELECT *
FROM read_files(
  'dbfs:/Volumes/course_ws_practise/default/olist/olist_orders_dataset.csv',
  format => 'csv',
  header => true,
  inferSchema => false
)

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.bricks_hub_olist.bronze_customers AS
SELECT *
FROM read_files(
  'dbfs:/Volumes/course_ws_practise/default/olist/olist_customers_dataset.csv',
  format => 'csv',
  header => true,
  inferSchema => false
)

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.bricks_hub_olist.bronze_order_items AS
SELECT *
FROM read_files(
  'dbfs:/Volumes/course_ws_practise/default/olist/olist_order_items_dataset.csv',
  format => 'csv',
  header => true,
  inferSchema => false
)

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.bricks_hub_olist.bronze_payments AS
SELECT *
FROM read_files(
  'dbfs:/Volumes/course_ws_practise/default/olist/olist_order_payments_dataset.csv',
  format => 'csv',
  header => true,
  inferSchema => false
);

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.bricks_hub_olist.bronze_products AS
SELECT *
FROM read_files(
  'dbfs:/Volumes/course_ws_practise/default/olist/olist_products_dataset.csv',
  format => 'csv',
  header => true,
  inferSchema => false
);

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.bricks_hub_olist.bronze_sellers AS
SELECT *
FROM read_files(
  'dbfs:/Volumes/course_ws_practise/default/olist/olist_sellers_dataset.csv',
  format => 'csv',
  header => true,
  inferSchema => false
);

In [0]:
%sql
SELECT * FROM workspace.bricks_hub_olist.bronze_sellers;

In [0]:
%sql
-- SILVER: orders table
CREATE OR REPLACE TABLE workspace.bricks_hub_olist.silver_orders AS
SELECT
  order_id,
  customer_id,
  lower(order_status)                         AS order_status,
  to_timestamp(order_purchase_timestamp)      AS order_purchase_ts,
  to_timestamp(order_approved_at)             AS order_approved_ts,
  to_timestamp(order_delivered_carrier_date)  AS order_delivered_carrier_ts,
  to_timestamp(order_delivered_customer_date) AS order_delivered_customers_ts,
  to_timestamp(order_estimated_delivery_date) AS order_estimated_delivery_ts
FROM workspace.bricks_hub_olist.bronze_orders

In [0]:
%sql
-- SILVER: customers table
CREATE OR REPLACE TABLE workspace.bricks_hub_olist.silver_customers AS
SELECT
  customer_id,
  customer_unique_id,
  customer_zip_code_prefix,
  upper(customer_city)     AS customer_city,
  upper(customer_state)    AS customer_state
FROM workspace.bricks_hub_olist.bronze_customers;

In [0]:
%sql
-- SILVER: order_items table
CREATE OR REPLACE TABLE workspace.bricks_hub_olist.silver_order_items AS
SELECT 
  order_id,
  CAST(order_item_id AS INT)     AS order_item_id,
  product_id,
  seller_id,
  CAST(price AS DOUBLE)          AS price,
  CAST(freight_value AS DOUBLE)  AS freight_value
FROM workspace.bricks_hub_olist.bronze_order_items;


In [0]:
%sql
-- SILVER: payments table
CREATE OR REPLACE TABLE workspace.bricks_hub_olist.silver_payments AS
SELECT
  order_id,
  CAST(payment_sequential AS INT)     AS payment_sequential,
  lower(payment_type)                 AS payment_type,
  CAST(payment_installments AS INT)   AS payment_installments,
  CAST(payment_value AS DOUBLE)       AS payment_value
FROM workspace.bricks_hub_olist.bronze_payments;

In [0]:
%sql
-- SILVER: products table
CREATE OR REPLACE TABLE workspace.bricks_hub_olist.silver_products AS
SELECT
  product_id,
  lower(product_category_name) AS product_category_name
FROM workspace.bricks_hub_olist.bronze_products;
  

In [0]:
%sql
-- SILVER: sellers table
CREATE OR REPLACE TABLE workspace.bricks_hub_olist.silver_sellers AS
SELECT
  seller_id,
  seller_zip_code_prefix,
  upper(seller_city) AS seller_city,
  upper(seller_state) AS seller_state
FROM workspace.bricks_hub_olist.bronze_sellers;

In [0]:
%sql
-- QA: missing IDs/FKs
SELECT 'orders_missing_id' AS check, COUNT(*) AS rows_missing FROM workspace.bricks_hub_olist.silver_orders WHERE order_id IS NULL;

In [0]:
%sql
SELECT 'customers_missing_id' AS check, COUNT(*) AS rows_missing FROM workspace.bricks_hub_olist.silver_customers WHERE customer_id IS NULL;

In [0]:
%sql
SELECT 'order_items_missing_fk' AS check, COUNT(*) AS rows_missing FROM workspace.bricks_hub_olist.silver_order_items WHERE order_id IS NULL;

In [0]:
%sql
-- QA: status distribution
SELECT order_status, COUNT(*) AS count
FROM workspace.bricks_hub_olist.silver_orders
GROUP BY order_status
ORDER BY count DESC;

In [0]:
%sql
-- QA: deivered date >= approved date
SELECT COUNT(*) AS bad_rows
FROM workspace.bricks_hub_olist.silver_orders
WHERE order_delivered_customers_ts IS NOT NULL
  AND order_approved_ts IS NOT NULL
  AND order_delivered_customers_ts < order_approved_ts;

In [0]:
%sql
-- EX 1: Top 5 most recent approved orders
SELECT order_id, customer_id, order_status, order_approved_ts
FROM workspace.bricks_hub_olist.silver_orders
WHERE order_approved_ts IS NOT NULL
ORDER BY order_approved_ts DESC
LIMIT 5;


In [0]:
%sql
-- EX 2: Orders missing delivery dates
SELECT order_id, order_status
FROM workspace.bricks_hub_olist.silver_orders
WHERE order_delivered_customers_ts IS NULL
LIMIT 10;

In [0]:
%sql
-- EX 3: Orders in a given state(join customers - SP)
SELECT o.order_id, c.customer_state, o.order_purchase_ts
FROM workspace.bricks_hub_olist.silver_orders o
JOIN workspace.bricks_hub_olist.silver_customers c
  ON o.customer_id = c.customer_id
WHERE c.customer_state = 'SP'
ORDER BY o.order_purchase_ts DESC
LIMIT 10;

In [0]:
%sql
-- EX 4: Most expensive order items (price)
SELECT order_id, product_id, price
FROM workspace.bricks_hub_olist.silver_order_items
ORDER BY price DESC
LIMIT 10;

In [0]:
%sql
-- EX 4: Most expensive order items (price)
SELECT order_id, product_id, price
FROM workspace.bricks_hub_olist.silver_order_items
ORDER BY price DESC
LIMIT 10;

In [0]:
%sql
-- EX 5: Join chain (Orders → Customers → Items → Products), sample 20
SELECT 
  o.order_id,
  c.customer_state,
  oi.product_id,
  p.product_category_name,
  oi.price,
  oi.freight_value
FROM workspace.bricks_hub_olist.silver_orders o
JOIN workspace.bricks_hub_olist.bronze_customers c
  ON o.customer_id = c.customer_id
JOIN workspace.bricks_hub_olist.silver_order_items oi
  ON o.order_id = oi.order_id
JOIN workspace.bricks_hub_olist.silver_products p
  ON oi.product_id = p.product_id
LIMIT 20;

In [0]:
%sql
-- KPI Prep: revenue per order view
CREATE OR REPLACE TABLE workspace.bricks_hub_olist.silver_order_revenue AS
SELECT 
  oi.order_id,
  SUM(oi.price + oi.freight_value) AS order_revenue
FROM workspace.bricks_hub_olist.silver_order_items oi
GROUP BY oi.order_id;

In [0]:
%sql
SELECT * FROM workspace.bricks_hub_olist.silver_order_revenue;

In [0]:
%sql
-- EX 6: Revenue by month
SELECT
  date_format(date_trunc('month', o.order_purchase_ts),  'yyyy-MMM' ) AS month,
  cast(SUM(r.order_revenue) AS decimal(10,2)) AS revenue
FROM workspace.bricks_hub_olist.silver_orders o
JOIN workspace.bricks_hub_olist.silver_order_revenue r
  ON o.order_id = r.order_id
GROUP BY 1
ORDER BY 1;

In [0]:
%sql
-- EX 7: Top 10 categories by revenue
SELECT
  coalesce(p.product_category_name, 'Unknown') AS category,
  CAST(SUM(oi.price + oi.freight_value) AS DECIMAL(10,2)) AS revenue
FROM workspace.bricks_hub_olist.silver_order_items oi
LEFT JOIN workspace.bricks_hub_olist.silver_products p
  ON oi.product_id = p.product_id
GROUP BY category
ORDER BY revenue DESC
--LIMIT 10;

In [0]:
%sql
-- EX 8: Payment mix
SELECT payment_type, COUNT(*) AS payments, CAST(SUM(payment_value) AS DECIMAL(10,2)) AS total_value
FROM workspace.bricks_hub_olist.silver_payments
GROUP BY payment_type
ORDER BY total_value DESC;

In [0]:
%sql
-- MINI Q1: Top 5 products by revenue
SELECT
  oi.product_id,
  coalesce(p.product_category_name, 'Unknown') AS category,
  CAST(SUM(oi.price + oi.freight_value) AS DECIMAL(10,2)) AS revenue
FROM workspace.bricks_hub_olist.silver_order_items oi
LEFT JOIN workspace.bricks_hub_olist.silver_products p
  ON oi.product_id = p.product_id
GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 5;

In [0]:
%sql
-- MINI Q2: Revenue by customer state (top 10)
SELECT
  c.customer_state,
  CAST(SUM(r.order_revenue) AS DECIMAL(10,2)) AS revenue
FROM workspace.bricks_hub_olist.silver_orders o
JOIN workspace.bricks_hub_olist.silver_customers c
  ON o.customer_id = c.customer_id
JOIN workspace.bricks_hub_olist.silver_order_revenue r
  ON o.order_id = r.order_id
GROUP BY 1
ORDER BY revenue DESC
LIMIT 10;

In [0]:
%sql
-- MINI Q3: Delivered order rate
SELECT
  ROUND(AVG(CASE WHEN order_delivered_customers_ts IS NOT NULL THEN 1.0 ELSE 0.0 END), 4) AS pct_delivered
FROM workspace.bricks_hub_olist.silver_orders;

In [0]:
%sql
-- GOLD: Revenue by month
CREATE OR REPLACE VIEW workspace.bricks_hub_olist.gold_revenue_by_month AS
SELECT
  date_format(date_trunc('month', o.order_purchase_ts),  'yyyy-MMM' ) AS month,
  cast(SUM(r.order_revenue) AS decimal(10,2)) AS revenue
FROM workspace.bricks_hub_olist.silver_orders o
JOIN workspace.bricks_hub_olist.silver_order_revenue r
  ON o.order_id = r.order_id
GROUP BY 1
ORDER BY 1;

In [0]:
%sql
-- GOLD: revenue by categories by
CREATE OR REPLACE VIEW workspace.bricks_hub_olist.gold_revenue_by_category AS
SELECT
  coalesce(p.product_category_name, 'Unknown') AS category,
  CAST(SUM(oi.price + oi.freight_value) AS DECIMAL(10,2)) AS revenue
FROM workspace.bricks_hub_olist.silver_order_items oi
LEFT JOIN workspace.bricks_hub_olist.silver_products p
  ON oi.product_id = p.product_id
GROUP BY category

In [0]:
%sql
-- GOLD: Payment mix (view)
CREATE OR REPLACE VIEW workspace.bricks_hub_olist.gold_payment_mix AS
SELECT payment_type, COUNT(*) AS payments, CAST(SUM(payment_value) AS DECIMAL(10,2)) AS total_value
FROM workspace.bricks_hub_olist.silver_payments
GROUP BY payment_type

In [0]:
%sql
-- Quick Check: order window
SELECT MIN(order_purchase_ts) AS first_order, MAX(order_purchase_ts) AS last_order
FROM workspace.bricks_hub_olist.silver_orders;

In [0]:
%sql
-- Quick Check: entity counts
SELECT
  (SELECT COUNT(DISTINCT customer_id) FROM workspace.bricks_hub_olist.silver_customers) AS customers,
  (SELECT COUNT(DISTINCT order_id) FROM workspace.bricks_hub_olist.silver_orders) AS orders,
  (SELECT COUNT(*)                  FROM workspace.bricks_hub_olist.silver_order_items) AS order_items;

In [0]:
%sql
-- Quick Check: AOV
SELECT ROUND(AVG(order_revenue),2) AS AOV
FROM workspace.bricks_hub_olist.silver_order_revenue;